In [2]:
import csv
import json

csv_file_path = './hack.csv'
json_file_path = './wordList.json'

data = []

with open(csv_file_path, mode='r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        data.append(row)

with open(json_file_path, mode='w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=2)


In [10]:
import json 
with open('./wordList.json', mode='r', encoding='utf-8') as json_file:
    word_list = json.load(json_file)


In [11]:
filtered_terms = [word['term'] for word in word_list if '(' in word['term'] or ' ' in word['term']]
print(filtered_terms)


['attributed to', 'ad hoc', 'aped bape', 'ascribe A to B', 'attest to', 'attest to', 'bereft of', 'by no means', 'clear out', 'clear out', 'contingent upon', 'convey ideas', 'disinclined to', 'dispense with', 'give way to', 'haggle about', 'hew to', 'in retrospect', 'jibe with', 'keep ~ at bay', 'make known', 'not fail to', 'on a par with', 'pull no punches', 'resign to', 'revert to', 'ring out', 'by no means', 'sifted from', 'stem from', 'take on']


In [12]:
word_list.sort(key=lambda x: x['term'])


In [13]:
word_list

[{'term': 'abash', 'definition': '[보통 수동형으로] 무안하게 하다, 당황하게 하다 (→abashed)'},
 {'term': 'abate', 'definition': '(정도, 세기 수량) 줄이다, 감소시키다, 완화시키다'},
 {'term': 'abet', 'definition': '(범죄, 나쁜 짓) 부추기다, 자극하다'},
 {'term': 'abeyance', 'definition': '(일시적) 중지, 중단'},
 {'term': 'abhorrence', 'definition': '(도덕적 이유에 의한) 혐오'},
 {'term': 'abhorrent', 'definition': '혐오스러운'},
 {'term': 'abhorrer', 'definition': '몹시 싫어하는 사람; (영국사) 국회 소집 반대파'},
 {'term': 'abject', 'definition': '1.비굴한 2.비참한, 절망적인'},
 {'term': 'abjure',
  'definition': '(신념, 주장) 버리다, 포기하다 \nrenounce, repudiate\nabdicate, relinquish'},
 {'term': 'abnegate', 'definition': '(권리) 버리다; (쾌락) 끊다, 자제하다'},
 {'term': 'abominate', 'definition': '혐오하다, 증오하다'},
 {'term': 'abominate', 'definition': 'despise, loath'},
 {'term': 'abound', 'definition': '풍부하다 \na. abundant'},
 {'term': 'abound', 'definition': 'proliferate'},
 {'term': 'absolve', 'definition': 'exornerate'},
 {'term': 'abstemious',
  'definition': '(음식, 음주) 삼가는, 절제하는 \n<abstain 삼가다 절제하다'},
 {

In [16]:
geo_file_path = './geo.txt'
csv_output_path = './geo.csv'

with open(geo_file_path, mode='r', encoding='utf-8') as geo_file:
    content = geo_file.read()

# Split the content into rows based on the specified row gap
rows = content.strip().split('\n\n\n\n\n')

# Prepare the CSV data
csv_data = []
for row in rows:
    parts = row.split('___%%%%%___')
    if len(parts) == 2:
        term = parts[0].strip()
        definition = parts[1].strip()
        csv_data.append({'term': term, 'definition': definition})

# Write to CSV
with open(csv_output_path, mode='w', encoding='utf-8', newline='') as csv_file:
    fieldnames = ['term', 'definition']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(csv_data)


In [18]:
import pandas as pd
df_geo = pd.read_csv('./geo.csv')

In [23]:
df_geo_json = df_geo.to_json(orient='records', force_ascii=False)
df_geo_json = json.loads(df_geo_json)  # Convert the JSON string to a Python object

In [24]:
import json

with open('geo.json', 'w', encoding='utf-8') as json_file:
    json.dump(df_geo_json, json_file, ensure_ascii=False, indent=4)


# Anki

In [155]:
import pandas as pd

# 입력 파일 경로 (예: 'gre_words.txt')
input_file = './anki.txt'
output_file = 'anki.csv'

# 파일 읽기
df = pd.read_csv(input_file, sep='\t', header=None, names=['term', 'definition'], quoting=3)




In [156]:
# df['raw'] = df['definition'].copy()


In [157]:
df['ipa'] = df['definition'].str.extract(r'/([^/]+)/')[0]
df['definition'] = df['definition'].str.replace(r'/[^/]+/', '', regex=True).str.strip()


In [158]:
# df['synonyms'] = df['definition'].str.extract(r'^(synonyms:|Similar:)\s*(.+?)(?=(?:"|\s{3,})\s*$)', expand=True)[1]


In [159]:
import re
def extract_all_synonyms(text):
    if not isinstance(text, str):
        return None  # 문자열이 아니면 처리 안함

    # 1. 앞부분 단어들 (n. v. 등으로 시작하기 전까지 , 로 구분된 구간)
    front_match = re.match(r'^([a-z ,\-]+)\s{3,}', text, flags=re.IGNORECASE)
    front = front_match.group(1).strip() if front_match else ''

    # 2. synonyms: 또는 Similar: 뒤의 모든 구간 추출
    pattern = r'(?:synonyms:|Similar:)\s*(.+?)(?=(?:"|\s{3,})\s*)'
    others = re.findall(pattern, text)

    # 3. 모두 모아서 리스트로 -> join
    all_terms = []
    if front:
        all_terms += [t.strip() for t in front.split(',')]
    for phrase in others:
        all_terms += [t.strip() for t in phrase.split(',')]

    return ', '.join([t for t in all_terms if t])


df['synonyms'] = df['definition'].apply(extract_all_synonyms)


In [163]:
import re
def extract_all_antonyms(text):
    if not isinstance(text, str):
        return None  # 문자열이 아니면 처리 안함

    # 1. 앞부분 단어들 (n. v. 등으로 시작하기 전까지 , 로 구분된 구간)
    front_match = re.match(r'^([a-z ,\-]+)\s{3,}', text, flags=re.IGNORECASE)
    front = front_match.group(1).strip() if front_match else ''

    pattern = r'(?:antonyms:|Opposite:)\s*(.+?)(?=(?:"|\s{3,})\s*)'
    others = re.findall(pattern, text)

    # 3. 모두 모아서 리스트로 -> join
    all_terms = []
    if front:
        all_terms += [t.strip() for t in front.split(',')]
    for phrase in others:
        all_terms += [t.strip() for t in phrase.split(',')]

    return ', '.join([t for t in all_terms if t])


df['antonyms'] = df['definition'].apply(extract_all_antonyms)


In [160]:
# df['antonyms'] = df['definition'].str.extract(r'^(?:.*?)(?:antonyms|Opposite):\s*(.+)')[0]


In [161]:
# import re
# def clean_synonyms(row):
#     if pd.notna(row['antonyms']):
#         pattern = rf"antonyms:\s*{re.escape(row['antonyms'])}"
#         return re.sub(pattern, "", row['synonyms']).strip()
#     return row['synonyms']

# df['synonyms'] = df.apply(clean_synonyms, axis=1)



In [167]:
# 2. raw 컬럼 복사
df['raw'] = df['definition']

# 3. 제거에 쓸 정규표현식 패턴들
# 앞부분 단어들 (n. v. 등 나오기 전까지)
front_pattern = r'^([a-z ,\-]+)\s{3,}'

# synonyms / Similar 패턴
syn_pattern = r'(?:synonyms:|Similar:)\s*(.+?)(?=(?:"|\s{3,})\s*)'

# antonyms / Opposite 패턴
ant_pattern = r'(?:antonyms:|Opposite:)\s*(.+?)(?=(?:"|\s{3,})\s*)'

# 4. 전처리 함수 정의
def clean_definition(text):
    if not isinstance(text, str):
        return text  # NaN은 그대로 유지

    text = re.sub(front_pattern, '', text, flags=re.IGNORECASE)
    text = re.sub(syn_pattern, '', text)
    text = re.sub(ant_pattern, '', text)
    return text.strip()

# 5. definition 컬럼에서 해당 패턴 제거
df['definition'] = df['definition'].apply(clean_definition)

In [169]:
df.columns

Index(['term', 'definition', 'ipa', 'synonyms', 'antonyms', 'raw'], dtype='object')

In [170]:
# 필요한 컬럼만 저장 (이미 필요한 컬럼만 있으므로 그대로 저장)
df[['term', 'definition', 'ipa', 'synonyms', 'antonyms']].to_csv(output_file, index=False)

In [172]:
import json
df = df[['term', 'definition', 'ipa', 'synonyms', 'antonyms']]
df_json = df.to_json(orient='records', force_ascii=False)
df_json = json.loads(df_json)
with open('anki.json', 'w', encoding='utf-8') as json_file:
    json.dump(df_json, json_file, ensure_ascii=False, indent=4)
